#### Create a priori trial parameter file (trialParam.priori.nc) ####
Given a list of to-be-evaluated parameters, create their corresponding a priori parameter values. 
1. update outputControl.txt by adding parameter names.
2. update fileManager.txt by changing simStartTime and simEndTime.
2. run SUMMA model to get a priori parameter values in summa output.
3. extract a priori parameter values from summa output and generate trialParam.priori.nc.

In [1]:
# import module
import os
import shutil
from datetime import datetime
import functions.utils as ut
import netCDF4 as nc
import sys
import numpy as np

In [2]:
# read paths from control_file
control_file = 'control_active.txt'
root_path = ut.read_from_control(control_file, 'root_path')
domain_name = ut.read_from_control(control_file, 'domain_name')
domain_path = os.path.join(root_path, domain_name)

In [3]:
# read new hydrologic model path
model_dst_path = ut.read_from_control(control_file, 'model_dst_path')
if model_dst_path == 'default':
    model_dst_path = os.path.join(domain_path, 'model')
summa_setting_path = os.path.join(model_dst_path, 'settings/SUMMA')

#### 1. Update outputControl.txt by adding parameter names.

In [4]:
object_params = ut.read_from_control(control_file, 'object_parameters')  # users provided object params
output_params = [x.strip() for x in object_params.split(',')]        # a more complete list of params that should be output in a priori parameter file 

# add more parameters if soil water content parameters are included in object_params.
if 'theta_sat' in object_params:
    for add_param in ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity']:
        if not add_param in object_params:
            output_params.append(add_param)            

# identify outputControl.txt and a temporary file.       
summa_outputControl = ut.read_from_control(control_file, 'summa_outputControl')
summa_outputControl_temp = summa_outputControl.split('.txt')[0]+'_temp.txt'

summa_outputControl = os.path.join(summa_setting_path, summa_outputControl)
summa_outputControl_temp = os.path.join(summa_setting_path, summa_outputControl_temp)

# add output_params to outputControl.txt            
with open(summa_outputControl, 'r') as src:
    content = src.read()
    with open(summa_outputControl_temp, 'w') as dst:
        for param in output_params:
            print(param, param in content)
            if not param in content:
                dst.write(param)
                dst.write('\n')
        dst.write(content)
shutil.copy2(summa_outputControl_temp, summa_outputControl);
os.remove(summa_outputControl_temp);

k_macropore True
k_soil True
theta_sat True
aquiferBaseflowExp True
aquiferBaseflowRate True
qSurfScale True
summerLAI True
frozenPrecipMultip True
heightCanopyBottom True
heightCanopyTop True
routingGammaScale True
routingGammaShape True
Fcapil True
theta_res True
critSoilWilting True
critSoilTranspire True
fieldCapacity True


#### 2. Update fileManager.txt by changing simStartTime and simEndTime. 

In [5]:
simStartTime_priori = ut.read_from_control(control_file, 'simStartTime_priori')
simEndTime_priori = ut.read_from_control(control_file, 'simEndTime_priori')

# identify fileManager.txt and a temporary file. 
summa_filemanager = ut.read_from_control(control_file, 'summa_filemanager')
summa_filemanager_temp = summa_filemanager.split('.txt')[0]+'_temp.txt'

summa_filemanager = os.path.join(summa_setting_path, summa_filemanager)
summa_filemanager_temp = os.path.join(summa_setting_path, summa_filemanager_temp)

# change sim times in fileManager.txt            
with open(summa_filemanager, 'r') as src:
    with open(summa_filemanager_temp, 'w') as dst:
        for line in src:
            if line.startswith('simStartTime'):
                simStartTime_old = line.split('!',1)[0].strip().split(None,1)[1]
                line = line.replace(simStartTime_old, simStartTime_priori)
            elif line.startswith('simEndTime'):
                simEndTime_old = line.split('!',1)[0].strip().split(None,1)[1]
                line = line.replace(simEndTime_old, simEndTime_priori)
            dst.write(line)
shutil.copy2(summa_filemanager_temp, summa_filemanager);
os.remove(summa_filemanager_temp);

#### 3. Run SUMMA model to get a priori parameter values in summa output.

In [6]:
# summa executable and settings paths 
summa_exe_path = ut.read_from_control(control_file, 'summa_exe_path')

# create summa output path if it does not exist.
with open(summa_filemanager, 'r') as src:
    for line in src:
        if line.startswith('outputPath'):
            outputPath = line.split('!',1)[0].strip().split(None,1)[1]
            outputPath = outputPath.strip("\'") # remove ' in string begin and end
            break
if not os.path.exists(outputPath):
    print('outputPath does not exist. Create it.')
    os.makedirs(outputPath)

# run SUMMA
cmd = summa_exe_path + ' -m '+ summa_filemanager
os.system(cmd)

outputPath does not exist. Create it.


0

#### 4. Extract a priori parameter values from summa output and generate trialParam.priori.nc.

In [7]:
# specify summa output, attribtue, and trialParam files
summa_ofile = os.path.join(outputPath, 'run1_timestep.nc')

summa_trialParams = ut.read_from_control(control_file, 'summa_trialParams')
summa_trialParams_priori =ut.read_from_control(control_file, 'summa_trialParams_priori') # a priori param file

summa_trialParams = os.path.join(summa_setting_path, summa_trialParams)
summa_trialParams_priori = os.path.join(summa_setting_path, summa_trialParams_priori)

summa_attributes = ut.read_from_control(control_file, 'summa_attributes')
summa_attributes = os.path.join(summa_setting_path, summa_attributes)

In [8]:
# open summa output file for reading
with nc.Dataset(summa_ofile, 'r') as ff:
    
    # if summa_trialParams does not exist, create summa_trialParams based on summa_attributes.
    if not os.path.exists(summa_trialParams):
        with nc.Dataset(summa_attributes) as src:
            with nc.Dataset(summa_trialParams, "w") as dst:

                # copy dimensions
                for name, dimension in src.dimensions.items():
                     dst.createDimension(
                        name, (len(dimension) if not dimension.isunlimited() else None))

                # copy gurId and hruId variables
                include = ['gruId', 'hruId']
                for name, variable in src.variables.items():
                    if name in include:
                        x = dst.createVariable(name, variable.datatype, variable.dimensions)               
                        dst[name].setncatts(src[name].__dict__)
                        dst[name][:]=src[name][:] 

                # create parameter varibles 
                for param_name in output_params:
                    param_value = ff[param_name][:].flat[0] # the first element of the array regardless dimensions                    
                    summa_ofile_dims = ff[param_name].dimensions
                    if 'hru' in summa_ofile_dims:
                        param_dim = 'hru'
                    elif 'gru' in summa_ofile_dims:
                        param_dim = 'gru'
                    else:
                        print('Variable %s is not in dimension gru or hru in summa output.'%(param_name))
                        sys.exit()

                    dst.createVariable(param_name, 'float', param_dim, fill_value=np.nan) 
                    dst[param_name][:] = param_value

    # if summa_trialParams exists, add to summa_trialParams based on summa_attributes.
    else:
        with nc.Dataset(summa_attributes) as src:
            with nc.Dataset(summa_trialParams, "w") as dst:

                # copy dimensions 
                for name, dimension in src.dimensions.items():
                     dst.createDimension(
                        name, (len(dimension) if not dimension.isunlimited() else None))

                # copy gurId and hruId variables
                include = ['gruId', 'hruId']
                for name, variable in src.variables.items():
                    if name in include:
                        x = dst.createVariable(name, variable.datatype, variable.dimensions)               
                        dst[name].setncatts(src[name].__dict__)
                        dst[name][:]=src[name][:] 

                # create parameter varibles 
                dst_vars=(dst.variables.keys()) # get all variable names of dst 
                for param_name in output_params:
                    param_value = ff[param_name][:].flat[0] # the first element of the array regardless dimensions                    
                    summa_ofile_dims = ff[param_name].dimensions
                    if 'hru' in summa_ofile_dims:
                        param_dim = 'hru'
                    elif 'gru' in summa_ofile_dims:
                        param_dim = 'gru'
                    else:
                        print('Variable %s is not in dimension gru or hru in summa output.'%(param_name))
                        sys.exit()

                    if not param_name in dst_vars:                    
                        dst.createVariable(param_name, 'float', param_dim, fill_value=np.nan) 
                        dst[param_name][:] = param_value

# copy summa_trialParams to get summa_trialParams_priori
shutil.copy2(summa_trialParams, summa_trialParams_priori);